### Interfaces/protocols and ABCs
#### 4 different ways of defining and using interfaces
1. Duck-typing
- Default approach since the beginning
- Runtime checking for structural types

2. Goose-typing
- Supported by ABCs which relies on **runtime** checks of objects against ABCs.
- run time check for nominal types

3. Static-typing
- traditional approach of statically-typed languages like C and Java; supported since Py3.5 by the typing module.
- **enforced by external-type-checkers**
- static checking for nominal types

4. Static duck typing
- Enforced by external-type checkers
- static checking for structural types


### Definition
1. structural types
types based on objects structure; methods provided by the object regardless of the name of its class or superclasses

2. nominal types
naming of the object's class or the name of it's superclasses

## Two Kinds of protocols
1. Networking protocol
2. An object protocol specifies methods which an object must provide to fulfill a role


In [2]:
class Vowels:
    data = [x for x in "AEIOU"]
    def __getitem__(self, i: int):
        return self.data[i]

In [3]:
v = Vowels()

In [4]:
# This works since __getitem__ special method is really 
# the key to sequence protocol
for c in v:
    print(c)

A
E
I
O
U


### 2 types of protocols
1. Dynamic Protocol
- the ifnromal protocols Python always had 
- They are implicit defined by convention and described in the documentation


2. Static protocol
- A protocl defined by Pep 544 (static duck-typing) .
- a typing.Protocol sub-class

There are 2 differencees between them
1. An object may implement only part of a dynamic protocol and still be useful; but to fulfill a static protocol, the object must provide every method declared in the protocol class
- static protocols can be verified by static type checkers but dynamic protocols can't

## Programming Ducks
- The philosophy of the Python Data Model is to cooperate wit h essential dynamic protocols as much as possible
- when it comes to sequences, Python tries hard to work with even the 
simples forms of implementations

## Monkey patching: implementing protocol at runtime


In [5]:
def set_vowel(vowels, position, alphabet):
    if position >= 5:
        raise ValueError('max position is 5')
    vowels.data[position] = alphabet
    
Vowels.__setitem__ = set_vowel

In [6]:
v[1] = 3

## Subclassing an ABC

In [8]:
from collections import namedtuple, abc

In [9]:
Card = namedtuple('Card', ['rank', 'suit'])

In [16]:
class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs heart'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
    
    def __delitem__(self, position):
        del self._cards[position]
    
    
    
    def insert(self, position, value):
        self._cards.insert(position, value)

In [17]:
deck2 = FrenchDeck2()

In [19]:
deck2[2]

Card(rank='4', suit='spades')

## Tombola ABC

In [24]:
import abc 
class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """
        Add items from an iterable
        """
    
    @abc.abstractmethod
    def pick(self):
        """
        Remove item at random and return
        
        Raises:
            Lookup error when the instance is empty
        """
    
    def loaded(self):
        """
        Return True if there is at least 1 item, 'False' otherwise
        """
    
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                    break
        self.load(items)
        return tuple(items)

## Static protocols

In [26]:
from typing import TypeVar, Protocol

In [27]:
T = TypeVar('T')

In [29]:
class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...

In [31]:
RT  = TypeVar('RT', bound=Repeatable)

In [32]:
def double(x: RT) -> RT:
    return x*2

## Designing a static protocol

In [38]:
from typing import Protocol, runtime_checkable, Any, TYPE_CHECKING, Iterable
import random

In [39]:
@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...

In [40]:
class SimplePicker:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)
    
    def pick(self) -> Any:
        return self._items.pop()


In [47]:
def test_isinstance() -> None:
    popper: RandomPicker = SimplePicker([1])
    assert isinstance(popper, RandomPicker)
    
def test_item_type() -> None:
    items = [1, 2]
    popper = SimplePicker(items) 
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)
    assert isinstance(item, int)

In [49]:
test_isinstance()

In [50]:
test_item_type()